# Tidy Data V2

**What are we trying to analyze?**

In the end, tidy data is contextual. There are some common methods to help reach a tidy dataset, but variable and observation definitions change depending on your goal. In general, a good process to understanding how to tidy your data may be:

1. Determine a question
2. Figure out which variables are needed to define your observation
3. Tidy your data
4. Answer your question

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

PLOT_TITLE_FONTDICT = {
    'fontsize': 'xx-large',
    'fontweight' : 'heavy'}

PLOT_AXIS_LABEL_FONTDICT = {
    'fontsize': 'x-large',
    'fontweight' : 'demibold'}

PLOT_TICK_FONTDICT = {
    'fontsize': 'large',
    'fontweight' : 'roman'}

In [ ]:
def create_dictionary_md_by_category(category):
    data_dictionary = pd.read_csv('https://extranet.who.int/tme/generateCSV.asp?ds=dictionary')
    data_dictionary = data_dictionary[data_dictionary['dataset'] == category] \
        .drop(columns=['dataset', 'code_list'])

    s = '## Data Dict\n\n'

    for var, definition in data_dictionary.values:
        s += f'### **{var}**\n{definition}\n\n'
        
    with open('./data_definition.md', 'w') as f:
        f.write(s)

In [ ]:
create_dictionary_md_by_category('Notification')

In [ ]:
tb_case_notifications = pd.read_csv('https://extranet.who.int/tme/generateCSV.asp?ds=notifications')
tb_case_notifications.head()

In [ ]:
'TB case data has {} rows and {} columns'.format(*tb_case_notifications.shape)

## The Data
This dataset comes from the World Health Organisation, and records the counts of confirmed tuberculosis cases by country, year, and demographic group. 
The demographic groups are broken down by sex (m, f) and age (0–14, 15–25, 25–34, 35–44, 45–54, 55–64, unknown).

> “Tuberculosis kills five thousand people every day. The social and economic impacts are devastating, including poverty, stigma and discrimination. While the world has committed to ending the TB epidemic by 2030, actions and investments don’t match the political rhetoric. We need a dynamic, global, multisectoral approach. The good news is that we finally have two great opportunities to move forward: the first WHO Global Ministerial Conference to End TB in Moscow in 2017, followed by the first UN General Assembly High-Level Meeting on TB in 2018. These will build momentum, get different sectors engaged, and accelerate our efforts to make TB history."

> Dr Tedros Adhanom Ghebreyesus, WHO Director-General

<hr>

## Step 1: Determine a question
`What gender and age bracket has the most common occurence of tuberculosis?`

<hr>

## Step 2: Figure out which `variables` are needed to define your `observation`
Ideally, you would want a data collection that would look like: 

| Gender (V) | Age Bracket (V) | Case Count (O) |
| - | - | - |
| M | 0-14 | 2 |
| F | 0-14 | 2 |
| .. | .. | .. |

* `Gender` - Categorical variable, either M or F. Excluding all unknowns
* `Age Bracket` - Categorical variable (0-14, 15-24, 25-34, 35-44, 45-54, 55-64, 65+)
* `Case Count` - Continuous observation, 0-n

### How is this data messy?
Well, a few ways!
* Plenty of unnecessary variables. 
We don't really need year or country. 
Additionally, we want to exclude all results where we don't have a definitive answer to either age or gender.
* The column names hold information about the variables!
Consider this: `new_sp_m514`

| TB type | Gender | Age Bracket |
| - | - | - |
| new_sp | m | 5-14 |

Ultimately, we will need to aggregate the counts for all types of TB, but that is still included in the column names.

<hr>

## Step 3: Tidy the data!

### Identifying the columns that hold our values.
Since we are not performing any sort of aggregation based on country/year, we can just target the case-specific columns.

In [ ]:
import re


def get_gender_age_bracket_specific_columns(df):
    pattern = re.compile(r'''
    (m|f)                              # has to include gender
    (014|1524|2534|3544|4554|5564|65)  # specific age brackets I am interested in
    ''', re.VERBOSE)
    
    return df.columns[df.columns.str.contains(pattern)]

In [ ]:
columns = get_gender_age_bracket_specific_columns(tb_case_notifications)
columns

In [ ]:
demo_tb_data_counts = tb_case_notifications[columns]
demo_tb_data_counts.head()

### Melting data into a manageable form
With all of these columns, it can be really hard to manage them all at once.
When the columns get melted, you can apply functionality to one column at a time and it affects all of the data.

In [ ]:
demo_tb_data_counts = demo_tb_data_counts.melt(var_name='demographics', value_name='tb_count')
demo_tb_data_counts

### Extract demographic data
We want to create 2 new columns that represent `gender` and `age_bracket` independent of one another.
Since the data for both variables live in the same column, we can use pandas built in `.str.extract` method to target each group, and create brand new columns by joining them together.
Additionally, we can drop the now useless `demographics` column.

In [ ]:
pattern = re.compile('''
.*
(?P<gender>[mf])      # named group, gender
(?P<age_bracket>\d+)  # named group, remaining digits
''', re.VERBOSE)


demo_tb_data_counts = demo_tb_data_counts \
    .join(demo_tb_data_counts['demographics'].str.extract(pattern, expand=True)) \
    .drop(columns=['demographics'])

demo_tb_data_counts

### Re-formatting age bracket for better read 
Adding hyphens or plus signs to better represent ranges.

In [ ]:
def reformat_age_bracket(ab):
    if ab == '014':
        return '0-14'
    elif ab == '65':
        return '65+'
    return ab[:2] + '-' + ab[2:]


demo_tb_data_counts['age_bracket'] = demo_tb_data_counts['age_bracket'].apply(reformat_age_bracket)
demo_tb_data_counts['age_bracket'].unique()

### Finally, we can group the demographics together to get a full count.

In [ ]:
demo_tb_data_counts = demo_tb_data_counts.groupby(['gender', 'age_bracket']) \
    ['tb_count'].sum() \
    .to_frame() \
    .reset_index()


demo_tb_data_counts

## Step 4: Answer your question
In this case, a grouped bar chart would show us exactly what we are wanting to know.

In [ ]:
# initialize fig/ax
fig, ax = plt.subplots(figsize=(16, 6), facecolor='lightyellow')

# some setup to help with labelling
bar_xticklabels = demo_tb_data_counts['age_bracket'].unique()
ind_positions = np.arange(len(bar_xticklabels))
bar_width = 0.35

# data to populate our plots
gender_groups = demo_tb_data_counts.groupby('gender')
age_bracket_counts = demo_tb_data_counts.groupby('age_bracket')['tb_count'].mean()

# set up bars
rect_1 = ax.bar(ind_positions-bar_width/2, gender_groups.get_group('m')['tb_count'], bar_width, label='Men', color='lightblue')
rect_2 = ax.bar(ind_positions+bar_width/2, gender_groups.get_group('f')['tb_count'], bar_width, label='Women', color='hotpink')

# line/scatter - same data
line1 = ax.plot(ind_positions, age_bracket_counts, color='purple')
scatter1 = ax.scatter(ind_positions, age_bracket_counts, color='purple', s=40, label='Avg Count', zorder=2)

# title
ax.set_title('TB Case Count by Gender and Age Bracket', fontdict=PLOT_TITLE_FONTDICT)

# y axis setup
ax.set_ylabel('TB Case Count', fontdict=PLOT_AXIS_LABEL_FONTDICT)
ax.set_yticklabels(['', '2M', '4M', '6M', '8M'], fontdict=PLOT_TICK_FONTDICT)

# x-axis setup
ax.set_xlabel('Age Brackets', fontdict=PLOT_AXIS_LABEL_FONTDICT)
ax.set_xticks(ind_positions)
ax.set_xticklabels(bar_xticklabels, fontdict=PLOT_TICK_FONTDICT)

# legend
ax.legend(title='Gender')

# additional styling
ax.set_facecolor("lightyellow")

# mark the highest with an annotation!
tb_max = demo_tb_data_counts['tb_count'].max()
ax.annotate(f'25-34 y/o Men with {int(tb_max):,} TB cases!',
            xy=(2, tb_max),  # theta, radius
            xytext=(0.85, 0.8),    # fraction, fraction
            textcoords='figure fraction',
            arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=5),
            horizontalalignment='right',
            verticalalignment='bottom',
            size=15
            )

plt.grid()
plt.show();